In [73]:

import csv
import os
import timeit

import argparse
import h5py
import logging
import numpy as np
import ufl

from dolfinx import cpp, fem, io, mesh
from mpi4py import MPI
from petsc4py import PETSc

import commons, configs, constants

markers = commons.SurfaceMarkers()

In [74]:
data_dir = "mesh/study_2/test6/470-470-5_000-000-000/"
tetpath = os.path.join(data_dir, "tetr.h5")
tripath = os.path.join(data_dir, "tria.h5")

In [75]:
geo2d = h5py.File(tripath, "r")
geo3d = h5py.File(tetpath, "r")

In [76]:
geo2d.keys(), geo3d.keys()

(<KeysViewHDF5 ['data0', 'data1', 'data2']>,
 <KeysViewHDF5 ['data0', 'data1', 'data2']>)

In [77]:
geo2d['data0'].shape, geo2d['data1'].shape, geo2d['data2'].shape

((420670, 3), (547158, 3), (547158,))

In [78]:
geo3d['data0'].shape, geo3d['data1'].shape, geo3d['data2'].shape

((420670, 3), (1738202, 4), (1738202,))

In [79]:
current3d = h5py.File("mesh/study_2/test6/470-470-5_000-000-000/current.h5", "r")

In [80]:
values = current3d['Function']['f']['0']

In [81]:
comm = MPI.COMM_WORLD
with io.XDMFFile(comm, tripath.replace(".h5", ".xdmf"), "r") as infile2:
    domain2 = infile2.read_mesh(cpp.mesh.GhostMode.none, 'Grid')
    ft = infile2.read_meshtags(domain2, name="Grid")
meshtags = mesh.meshtags(domain2, 2, ft.indices, ft.values)

In [82]:
W = fem.FunctionSpace(domain2, ("Lagrange", 1))
current = fem.Function(W)

In [83]:
valz = set(np.asarray(geo2d['data1']).flatten())
for dof in valz:
    current.vector.setValueLocal(dof, values[dof][2])